# Supervised Coordinate Classification
In this notebook we compare performance of Convolutional Layers with CoordConv Layers, introduced in paper - [An intriguing failing of convolutional neural networks and the CoordConv solution](https://arxiv.org/pdf/1807.03247.pdf).

## Problem Statement
Compare classification performance of Convolution layer with CoordConv layer on the *NotSoClever* dataset introduced in the paper.

### Not So Clever Dataset
Not-so-Clevr consists of 9×9 squares placed on a 64×64 canvas. Square positions are restricted such that the entire square lies within the 64×64 grid, so that square centers fall within a slightly smaller possible area of 56×56. Enumerating these
possible center positions results in a dataset with a total of 3,136 examples. For each example square i, the dataset contains three fields:

* $C_{i} \in \mathbb{R}^2$, its center location in (x,y) Cartesian coordinates,
* $P_{i} \in \mathbb{R}^{64 \times 64}$, a one-hot representation of its center pixel, and
* $I_{i} \in \mathbb{R}^{64 \times 64}$, the resulting 64×64 image of the square painted on the canvas.

### Train/Test Split
We will go for the classification problem and will work with two kind of train/test split:

* Quadrant Split
* Uniform Split

![NotSoClever Dataset](./images/notsoclever_ds.png)

## CoordConv Layer
CoordConv is an extension to Convoluton layers just to give the neural network extra understanding of the coordinates of the pixels. The layer keeps two properties of Convolution layer intact:
* few parameters
* efficient computation

But it gives a degree of freedom to network in case of *translation invariance*. It lets the network to decide whether it needs the said property or not.

|![CoordConv Vs Conv](./images/coordconv_vs_conv.png)|
|:--------------------------------------------------:|
|Source: https://arxiv.org/pdf/1807.03247 |

Run the following cell to make sure our working directory is ready to save the models we train.

In [1]:
import os

if not os.path.isdir('./pixel_classification'):
    os.makedirs('./pixel_classification')

## Deconvolution over NotSoClever - Quadrant Split
|![Deconvolution Architecture for NotSoClever](./images/deconv_arch_clf.png)|
|:-------------------------------:|
| Source: https://arxiv.org/pdf/1807.03247 |

* fs = {2, 3, 4}
* c = {1, 2, 3}

In [1]:
import tensorflow as tf

from utils.nsc import NotSoCleverCreator as NSCCreator
from models.supervised_conv import model_fn as conv_model_fn
from nn.pipeline import classifier_input_fn

In [2]:
nsc = NSCCreator()

X_train, X_test = nsc.quadrant_split()
Y_train = nsc.one_hot_coords(nsc.canvas_size, X_train)
Y_test = nsc.one_hot_coords(nsc.canvas_size, X_test)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((2408, 2), (841, 2), (2408, 64, 64, 1), (841, 64, 64, 1))

In [3]:
estimator = tf.estimator.Estimator(model_fn=conv_model_fn,
                                   params={
                                       'lr': 1e-3,
                                       'c': 1
                                   },
                                   model_dir='./pixel_classification/supervised_quad_conv')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_task_type': 'worker', '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_train_distribute': None, '_tf_random_seed': None, '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022C49C02FD0>, '_session_config': None, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_is_chief': True, '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_master': '', '_service': None, '_model_dir': './pixel_classification/supervised_quad_conv'}


In [4]:
train_in_fn = lambda: classifier_input_fn(X_train, Y_train, 32)
test_in_fn = lambda: classifier_input_fn(X_test, Y_test, 32, is_train=False)

In [5]:
EPOCHS = 100
STEPS_PER_EPOCH = X_train.shape[0] // 32
STEPS = EPOCHS * STEPS_PER_EPOCH

STEPS

7500

In [6]:
estimator.train(input_fn=train_in_fn, steps=STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./pixel_classification/supervised_quad_conv\model.ckpt.
INFO:tensorflow:step = 0, loss = 8.31728
INFO:tensorflow:accuracy = 0.0
INFO:tensorflow:global_step/sec: 24.6763
INFO:tensorflow:step = 100, loss = 7.8248906 (4.068 sec)
INFO:tensorflow:accuracy = 0.03125 (4.052 sec)
INFO:tensorflow:global_step/sec: 35.8465
INFO:tensorflow:step = 200, loss = 6.0391746 (2.774 sec)
INFO:tensorflow:accuracy = 0.0 (2.774 sec)
INFO:tensorflow:global_step/sec: 36.2574
INFO:tensorflow:step = 300, loss = 5.714446 (2.758 sec)
INFO:tensorflow:accuracy = 0.0 (2.758 sec)
INFO:tensorflow:global_step/sec: 36.325
INFO:tensorflow:step = 400, loss = 5.3855057 (2.769 sec)
INFO:tensorflow:accuracy = 0.03125 (2.769 sec)
INFO:tensorflow:glo

In [7]:
estimator.evaluate(input_fn=lambda: classifier_input_fn(X_train,
                                                        Y_train,
                                                        32, is_train=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-05-09:28:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./pixel_classification/supervised_quad_conv\model.ckpt-7500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:accuracy = 1.0
INFO:tensorflow:Finished evaluation at 2018-08-05-09:28:58
INFO:tensorflow:Saving dict for global step 7500: global_step = 7500, loss = 0.7289746


{'global_step': 7500, 'loss': 0.7289746}

In [8]:
estimator.evaluate(input_fn=test_in_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-05-09:29:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./pixel_classification/supervised_quad_conv\model.ckpt-7500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:accuracy = 0.0
INFO:tensorflow:Finished evaluation at 2018-08-05-09:29:16
INFO:tensorflow:Saving dict for global step 7500: global_step = 7500, loss = 7310.641


{'global_step': 7500, 'loss': 7310.641}

## CoordConv over NotSoClever - Quadrant Split
|![CoordConv Architecture for NotSoClever](./images/coordconv_arch_clf.png)|
|:-------------------------------:|
| Source: https://arxiv.org/pdf/1807.03247 |

In [1]:
import tensorflow as tf

from utils.nsc import NotSoCleverCreator as NSCCreator
from models.supervised_coordconv import model_fn as coordconv_model_fn
from nn.pipeline import classifier_input_fn

In [2]:
nsc = NSCCreator()

X_train, X_test = nsc.quadrant_split()
Y_train = nsc.one_hot_coords(nsc.canvas_size, X_train)
Y_test = nsc.one_hot_coords(nsc.canvas_size, X_test)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((2408, 2), (841, 2), (2408, 64, 64, 1), (841, 64, 64, 1))

In [3]:
estimator = tf.estimator.Estimator(model_fn=coordconv_model_fn,
                                   params={
                                       'lr': 1e-3
                                   },
                                   model_dir='./pixel_classification/supervised_quad_coord')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_is_chief': True, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E915E859B0>, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_global_id_in_cluster': 0, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_log_step_count_steps': 100, '_session_config': None, '_task_id': 0, '_num_worker_replicas': 1, '_tf_random_seed': None, '_model_dir': './pixel_classification/supervised_quad_coord', '_task_type': 'worker', '_save_checkpoints_steps': None, '_train_distribute': None}


In [4]:
train_in_fn = lambda: classifier_input_fn(X_train, Y_train, 32)
test_in_fn = lambda: classifier_input_fn(X_test, Y_test, 32, is_train=False)

In [5]:
EPOCHS = 100
STEPS_PER_EPOCH = X_train.shape[0] // 32
STEPS = EPOCHS * STEPS_PER_EPOCH

STEPS

7500

In [6]:
estimator.train(input_fn=train_in_fn, steps=STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./pixel_classification/supervised_quad_coord\model.ckpt.
INFO:tensorflow:step = 0, loss = 8.287167
INFO:tensorflow:accuracy = 0.0
INFO:tensorflow:global_step/sec: 3.42047
INFO:tensorflow:step = 100, loss = 7.5643306 (29.274 sec)
INFO:tensorflow:accuracy = 0.0 (29.274 sec)
INFO:tensorflow:global_step/sec: 3.57952
INFO:tensorflow:step = 200, loss = 6.385539 (27.899 sec)
INFO:tensorflow:accuracy = 0.0 (27.899 sec)
INFO:tensorflow:global_step/sec: 3.58036
INFO:tensorflow:step = 300, loss = 5.1956162 (27.993 sec)
INFO:tensorflow:accuracy = 0.0625 (27.993 sec)
INFO:tensorflow:global_step/sec: 3.60377
INFO:tensorflow:step = 400, loss = 4.6345787 (27.702 sec)
INFO:tensorflow:accuracy = 0.03125 (27.702 sec)
INFO:tens

In [7]:
estimator.evaluate(input_fn=lambda: classifier_input_fn(X_train,
                                                        Y_train,
                                                        32, is_train=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-05-10:23:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./pixel_classification/supervised_quad_coord\model.ckpt-7500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:accuracy = 0.40625
INFO:tensorflow:Finished evaluation at 2018-08-05-10:23:39
INFO:tensorflow:Saving dict for global step 7500: global_step = 7500, loss = 2.7045329


{'global_step': 7500, 'loss': 2.7045329}

In [8]:
estimator.evaluate(input_fn=test_in_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-05-10:23:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./pixel_classification/supervised_quad_coord\model.ckpt-7500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:accuracy = 0.21875
INFO:tensorflow:Finished evaluation at 2018-08-05-10:23:53
INFO:tensorflow:Saving dict for global step 7500: global_step = 7500, loss = 2.587729


{'global_step': 7500, 'loss': 2.587729}

## Deconvolution over NotSoClever - Uniform Split
*Same architecture*

In [1]:
import tensorflow as tf

from utils.nsc import NotSoCleverCreator as NSCCreator
from models.supervised_conv import model_fn as conv_model_fn
from nn.pipeline import classifier_input_fn

In [2]:
nsc = NSCCreator()

X_train, X_test = nsc.uniform_split()
Y_train = nsc.one_hot_coords(nsc.canvas_size, X_train)
Y_test = nsc.one_hot_coords(nsc.canvas_size, X_test)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((2436, 2), (813, 2), (2436, 64, 64, 1), (813, 64, 64, 1))

In [3]:
estimator = tf.estimator.Estimator(model_fn=conv_model_fn,
                                   params={
                                       'lr': 1e-3,
                                       'c': 1
                                   },
                                   model_dir='./pixel_classification/supervised_uniform_conv')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './pixel_classification/supervised_uniform_conv', '_session_config': None, '_log_step_count_steps': 100, '_train_distribute': None, '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A67D022E10>, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_is_chief': True, '_global_id_in_cluster': 0, '_save_summary_steps': 100, '_task_type': 'worker', '_service': None, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_evaluation_master': '', '_task_id': 0}


In [4]:
train_in_fn = lambda: classifier_input_fn(X_train, Y_train, 32)
test_in_fn = lambda: classifier_input_fn(X_test, Y_test, 32, is_train=False)

In [5]:
EPOCHS = 100
STEPS_PER_EPOCH = X_train.shape[0] // 32
STEPS = EPOCHS * STEPS_PER_EPOCH

STEPS

7600

In [6]:
estimator.train(input_fn=train_in_fn, steps=STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./pixel_classification/supervised_uniform_conv\model.ckpt.
INFO:tensorflow:loss = 8.315047, step = 0
INFO:tensorflow:accuracy = 0.0
INFO:tensorflow:global_step/sec: 24.9472
INFO:tensorflow:loss = 7.67513, step = 100 (4.008 sec)
INFO:tensorflow:accuracy = 0.0 (4.025 sec)
INFO:tensorflow:global_step/sec: 36.4496
INFO:tensorflow:loss = 6.647333, step = 200 (2.744 sec)
INFO:tensorflow:accuracy = 0.0 (2.727 sec)
INFO:tensorflow:global_step/sec: 36.553
INFO:tensorflow:loss = 5.9990826, step = 300 (2.736 sec)
INFO:tensorflow:accuracy = 0.0 (2.736 sec)
INFO:tensorflow:global_step/sec: 36.9695
INFO:tensorflow:loss = 5.4760075, step = 400 (2.705 sec)
INFO:tensorflow:accuracy = 0.0 (2.721 sec)
INFO:tensorflow:global_st

In [7]:
estimator.evaluate(input_fn=lambda: classifier_input_fn(X_train,
                                                        Y_train,
                                                        32, is_train=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-05-10:51:31
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./pixel_classification/supervised_uniform_conv\model.ckpt-7600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:accuracy = 0.78125
INFO:tensorflow:Finished evaluation at 2018-08-05-10:51:42
INFO:tensorflow:Saving dict for global step 7600: global_step = 7600, loss = 0.5717341


{'global_step': 7600, 'loss': 0.5717341}

In [8]:
estimator.evaluate(input_fn=test_in_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-05-10:51:51
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./pixel_classification/supervised_uniform_conv\model.ckpt-7600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:accuracy = 0.40625
INFO:tensorflow:Finished evaluation at 2018-08-05-10:51:55
INFO:tensorflow:Saving dict for global step 7600: global_step = 7600, loss = 0.9549831


{'global_step': 7600, 'loss': 0.9549831}

## CoordConv over NotSoClever - Uniform Split
*Same Architecture*

In [1]:
import tensorflow as tf

from utils.nsc import NotSoCleverCreator as NSCCreator
from models.supervised_coordconv import model_fn as coordconv_model_fn
from nn.pipeline import classifier_input_fn

In [2]:
nsc = NSCCreator()

X_train, X_test = nsc.uniform_split()
Y_train = nsc.one_hot_coords(nsc.canvas_size, X_train)
Y_test = nsc.one_hot_coords(nsc.canvas_size, X_test)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((2436, 2), (813, 2), (2436, 64, 64, 1), (813, 64, 64, 1))

In [3]:
estimator = tf.estimator.Estimator(model_fn=coordconv_model_fn,
                                   params={
                                       'lr': 1e-2
                                   },
                                   model_dir='./pixel_classification/supervised_uniform_coord')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_task_id': 0, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000024F62723D68>, '_num_worker_replicas': 1, '_master': '', '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_session_config': None, '_tf_random_seed': None, '_is_chief': True, '_service': None, '_global_id_in_cluster': 0, '_model_dir': './pixel_classification/supervised_uniform_coord', '_evaluation_master': ''}


In [4]:
train_in_fn = lambda: classifier_input_fn(X_train, Y_train, 32)
test_in_fn = lambda: classifier_input_fn(X_test, Y_test, 32, is_train=False)

In [5]:
EPOCHS = 100
STEPS_PER_EPOCH = X_train.shape[0] // 32
STEPS = EPOCHS * STEPS_PER_EPOCH

STEPS

7600

In [ ]:
estimator.train(input_fn=train_in_fn, steps=STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./pixel_classification/supervised_uniform_coord\model.ckpt.
INFO:tensorflow:step = 0, loss = 8.337212
INFO:tensorflow:accuracy = 0.0
INFO:tensorflow:global_step/sec: 3.41602
INFO:tensorflow:step = 100, loss = 7.7774725 (29.291 sec)
INFO:tensorflow:accuracy = 0.0 (29.275 sec)
INFO:tensorflow:global_step/sec: 3.58353
INFO:tensorflow:step = 200, loss = 6.0056696 (27.916 sec)
INFO:tensorflow:accuracy = 0.0 (27.917 sec)
INFO:tensorflow:global_step/sec: 3.57938
INFO:tensorflow:step = 300, loss = 6.148604 (27.914 sec)
INFO:tensorflow:accuracy = 0.0 (27.915 sec)
INFO:tensorflow:global_step/sec: 3.60666
INFO:tensorflow:step = 400, loss = 6.002205 (27.723 sec)
INFO:tensorflow:accuracy = 0.0 (27.720 sec)
INFO:tensorflo

In [6]:
estimator.evaluate(input_fn=lambda: classifier_input_fn(X_train,
                                                        Y_train,
                                                        32, is_train=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-05-11:30:53
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./pixel_classification/supervised_uniform_coord\model.ckpt-4259
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:accuracy = 1.0
INFO:tensorflow:Finished evaluation at 2018-08-05-11:31:14
INFO:tensorflow:Saving dict for global step 4259: global_step = 4259, loss = 0.004766937


{'global_step': 4259, 'loss': 0.004766937}

In [7]:
estimator.evaluate(input_fn=test_in_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-05-11:31:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./pixel_classification/supervised_uniform_coord\model.ckpt-4259
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:accuracy = 1.0
INFO:tensorflow:Finished evaluation at 2018-08-05-11:31:28
INFO:tensorflow:Saving dict for global step 4259: global_step = 4259, loss = 0.0046338784


{'global_step': 4259, 'loss': 0.0046338784}